# Color-CIFAR10-Shift Demo

In [ ]:
!pip install -q git+https://github.com/lae-um/laplace-torch@main
import datasets
from laplace import Laplace
from torchvision.models import wide_resnet50_2
import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader
from PIL import Image

ds = datasets.load_dataset('color-cifar10-shift')
test_imgs = [Image.open(f) for f in ds['test_control']['image']]
test_tensors = torch.stack([T.ToTensor()(im) for im in test_imgs])
test_loader = DataLoader([(img, lbl) for img, lbl in zip(test_tensors, ds['test_control']['label'])], batch_size=128)
model = wide_resnet50_2(pretrained=True).eval()
la = Laplace(model, 'classification', subset_of_weights='last_layer', hessian_structure='kron')
la.fit(test_loader)
la.optimize_prior_precision()
print('Laplace ready!')